
# Analyze Mean Fluorescence Intensity (MFI) — *nodox* images

This notebook scans one or more folders for `.czi` **Z-stack** images whose filenames contain **`nodox`** (case‑insensitive), computes:

- the **mean intensity per z-slice** of the **orange/TMRM channel (channel 0)**, then  
- the **average across z-slices** for each image,

and compiles one **Excel spreadsheet per folder** with two columns:
`Image`, `Orange MFI`.

> **Filters (defaults):**
> - filename must include `zstack`
> - filename must not include `mip`
> - filename must include `nodox`


In [21]:

# --- Imports ---
import os
import traceback
import numpy as np
import pandas as pd

# CZI reader
from aicspylibczi import CziFile


## Configure folders and filters

In [22]:

# Edit this list to your folders (or override in the Run cell below)
FOLDERS = [
    '090525_1k_WT_dox_TMRM+Hoeschst',
    '091225_1k_wt_dox_TMRM_HOESCHST',
    '091225_1k_wt_dox_TMRM_HOESCHST/wt'
]

# Filters
FILTER_REQUIRE_ZSTACK = True   # include only filenames containing 'zstack'
FILTER_EXCLUDE_MIP    = True   # exclude filenames containing 'mip'
REQUIRE_SUBSTRING     = ''  # include only filenames containing this substring (case-insensitive)


## Helper functions

In [23]:

def is_valid_image_name(filename):
    """Apply filename filters to match your workflow."""
    name = filename.lower()
    if not name.endswith('.czi'):
        return False
    if REQUIRE_SUBSTRING and (REQUIRE_SUBSTRING.lower() not in name):
        return False
    if FILTER_REQUIRE_ZSTACK and ('zstack' not in name):
        return False
    if FILTER_EXCLUDE_MIP and ('mip' in name):
        return False
    return True


def compute_orange_mfi_across_z(czi_path):
    """
    Load CZI, take channel 0 as the orange/TMRM stack (Z, Y, X),
    compute mean per Z, then return the average across Z as a float.
    """
    cz = CziFile(czi_path)
    arr = np.squeeze(cz.read_image(S=1)[0])  # ndarray

    # Expect shapes like (C, Z, Y, X) or (Z, Y, X).
    if arr.ndim < 3:
        raise ValueError(f"Unexpected array ndims={arr.ndim} for {os.path.basename(czi_path)}")

    if arr.ndim == 4:
        # (C, Z, Y, X)
        if arr.shape[0] < 1:
            raise ValueError("First dimension (channels) < 1")
        orange = arr[0]  # (Z, Y, X)
    elif arr.ndim == 3:
        # Already single-channel (Z, Y, X)
        orange = arr
    else:
        raise ValueError(f"Unhandled array shape {arr.shape}")

    # Handle single-plane case uniformly
    if orange.ndim == 2:
        per_z_means = [float(np.mean(orange))]
    elif orange.ndim == 3:
        per_z_means = [float(np.mean(orange[z])) for z in range(orange.shape[0])]
    else:
        raise ValueError(f"Unhandled orange shape {orange.shape}")

    return float(np.mean(per_z_means))


def analyze_folder(folder, verbose=True):
    """Scan a folder (non-recursive), compute MFI for each matching .czi, save an Excel file."""
    if not os.path.isdir(folder):
        if verbose:
            print(f"[skip] Folder does not exist: {folder}")
        return None

    rows = []
    files = sorted(os.listdir(folder))
    if verbose:
        print(f"\n=== Scanning: {folder} ===")

    for fname in files:
        if not is_valid_image_name(fname):
            continue
        fpath = os.path.join(folder, fname)
        try:
            mfi = compute_orange_mfi_across_z(fpath)
            rows.append({"Image": os.path.splitext(fname)[0], "Orange MFI": mfi})
            if verbose:
                print(f"  ✓ {fname}: {mfi:.6f}")
        except Exception as e:
            print(f"  ✗ {fname}: ERROR -> {e}")
            if verbose:
                traceback.print_exc()

    if not rows:
        if verbose:
            print("[info] No matching images found or no valid data extracted.")
        return None

    df = pd.DataFrame(rows).sort_values("Image").reset_index(drop=True)

    # Output file (one per folder)
    safe_name = folder.replace('/', '_').replace('\\', '_').replace(' ', '_').strip('_')
    out_xlsx = os.path.join(folder, f"{safe_name}_mfi_whole_image.xlsx")
    os.makedirs(folder, exist_ok=True)  # in case the folder path doesn't exist yet in this environment
    df.to_excel(out_xlsx, index=False)

    if verbose:
        print(f"\n✅ Saved: {out_xlsx}\n")
    return out_xlsx


## Run analysis

In [24]:

# You can modify FOLDERS here before running.
# Example:
# FOLDERS = ['path/to/folderA', 'path/to/folderB']
# folders =  ['72125_3K_dox_TMRM_hoechst/new settings','72525_1K_WT_dox_TMRM_hoechst','72825_3K_dox_TMRM_hoechst','80425_3K_dox_TMRM_hoechst','080825_3K_dox_TMRM_hoechst']
folders = ['081225_3K_1K_WT_dox_TMRM_hoechst','081225_3K_1K_WT_dox_TMRM_hoechst/1K','081225_3K_1K_WT_dox_TMRM_hoechst/WT new settings','082025_1K_dox_TMRM_Hoeschst','082625_1k_dox_TMRM_Hoeschst','090225_1k_dox_TMRM_Hoeschst']
folders = ['082025_1K_dox_TMRM_Hoeschst/WT','082625_1k_dox_TMRM_Hoeschst/NEW SETTINGS/1K_TMRM_hoechst','090225_1k_dox_TMRM_Hoeschst/NEW SETTINGS',
           '090225_1k_dox_TMRM_Hoeschst/NEW SETTINGS/WT','090425_1k_WT_dox_TMRM+Hoeschst','090425_1k_WT_dox_TMRM+Hoeschst/WT','090425_1k_WT_dox_TMRM+Hoeschst',
           '090925_1k_wt_dox_TMRM_HOESCHST','090925_1k_wt_dox_TMRM_HOESCHST/WT']
folders = ['090525_1k_WT_dox_TMRM+Hoeschst','091225_1k_wt_dox_TMRM_HOESCHST','091225_1k_wt_dox_TMRM_HOESCHST/wt']
outputs = []
for _folder in folders:
    out = analyze_folder(_folder, verbose=True)
    outputs.append((_folder, out))

outputs



=== Scanning: 090525_1k_WT_dox_TMRM+Hoeschst ===
  ✓ 1K_NOdox_OA_TMRM_hoechst_zstack_01.czi: 940.483051
  ✓ 1K_NOdox_OA_TMRM_hoechst_zstack_02.czi: 1603.634431
  ✓ 1K_NOdox_OA_TMRM_hoechst_zstack_03.czi: 837.827231
  ✓ 1K_NOdox_OA_TMRM_hoechst_zstack_04.czi: 554.048072
  ✓ 1K_NOdox_OA_TMRM_hoechst_zstack_05.czi: 1017.304315
  ✓ 1K_NOdox_OA_TMRM_hoechst_zstack_06.czi: 485.437357
  ✓ 1K_NOdox_OA_TMRM_hoechst_zstack_07.czi: 507.994912
  ✓ 1K_NOdox_OA_TMRM_hoechst_zstack_08.czi: 210.879099
  ✓ 1K_dox_OA_TMRM_hoechst_CCCP_zstack_01.czi: 457.295079
  ✓ 1K_dox_OA_TMRM_hoechst_CCCP_zstack_02.czi: 347.502869
  ✓ 1K_dox_OA_TMRM_hoechst_CCCP_zstack_03.czi: 306.031276
  ✓ 1K_dox_OA_TMRM_hoechst_CCCP_zstack_04.czi: 629.029957
  ✓ 1K_dox_OA_TMRM_hoechst_zstack_01.czi: 785.995633
  ✓ 1K_dox_OA_TMRM_hoechst_zstack_02.czi: 1296.746596
  ✓ 1K_dox_OA_TMRM_hoechst_zstack_03.czi: 651.477927
  ✓ 1K_dox_OA_TMRM_hoechst_zstack_04.czi: 1407.104390
  ✓ 1K_dox_OA_TMRM_hoechst_zstack_05.czi: 498.171232
  ✓ 1K_do

[('090525_1k_WT_dox_TMRM+Hoeschst',
  '090525_1k_WT_dox_TMRM+Hoeschst\\090525_1k_WT_dox_TMRM+Hoeschst_mfi_whole_image.xlsx'),
 ('091225_1k_wt_dox_TMRM_HOESCHST',
  '091225_1k_wt_dox_TMRM_HOESCHST\\091225_1k_wt_dox_TMRM_HOESCHST_mfi_whole_image.xlsx'),
 ('091225_1k_wt_dox_TMRM_HOESCHST/wt',
  '091225_1k_wt_dox_TMRM_HOESCHST/wt\\091225_1k_wt_dox_TMRM_HOESCHST_wt_mfi_whole_image.xlsx')]